# Fully connected NN

In this script, we are implementing a fully connected, 3-layers NN with the following layers:

* __DropOut__ for regularization
* __BatchNormalization__ for effective optimization
* __ReLU__ activation function
* __SoftMax/SVM__ as loss function
* __RMSProp/Adam__ as optimizer

In [61]:
from __future__ import print_function

import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

In [62]:
### load sample of the data and split it to training and testing data
def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

batch_1 = unpickle('../data/cifar-10-batches-py/data_batch_1')
data = batch_1['data']
labels = batch_1['labels']

train_data = data[:7000,:]
test_data = data[7000:,:]
y_train = labels[:7000]
y_test = labels[7000:]

print("Train data shape: {}".format(train_data.shape))
print("Test data shape: {}".format(test_data.shape))




Train data shape: (7000, 3072)
Test data shape: (3000, 3072)


In [87]:
### Normalize data and get batches of tensors
means = train_data.mean(axis=0)
stds = train_data.std(axis=0)

X_train = (train_data - means)/stds
X_test = (test_data - means)/stds

train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
train_loader = iter(torch.utils.data.DataLoader(train_dataset, batch_size=8))

test_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
test_loader = iter(torch.utils.data.DataLoader(test_dataset, batch_size=8))

In [98]:
### define fully-connected NN
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, layers_sizes=[3072, 1024, 516, 10]):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(in_features=layers_sizes[0], out_features=layers_sizes[1])
        self.fc2 = nn.Linear(in_features=layers_sizes[1], out_features=layers_sizes[2])
        self.fc3 = nn.Linear(in_features=layers_sizes[2], out_features=10)
        self.bn = nn.BatchNorm1d(num_features=layers_sizes[1])
        
    def forward(self, x):
        x = F.dropout(F.relu(self.fc1(x)), p=0.2)
        x = self.bn(x)
        x = F.dropout(F.relu(self.fc2(x)), p=0.2)
        y_pred = self.fc3(x)
        return y_pred
        
net = Net()

In [99]:
### define loss criteria and optimizer
import torch.optim as optim

cross_entropy = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [100]:
for epoch in range(10):  # loop over the dataset multiple times

    train_loader = iter(torch.utils.data.DataLoader(train_dataset, batch_size=8))
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs).type(torch.FloatTensor), Variable(labels).type(torch.LongTensor)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
print('Finished Training')

[1,   100] loss: 2.209
[1,   200] loss: 2.085
[1,   300] loss: 2.009
[1,   400] loss: 1.999
[1,   500] loss: 1.930
[1,   600] loss: 1.938
[1,   700] loss: 1.966
[1,   800] loss: 1.858
[2,   100] loss: 1.844
[2,   200] loss: 1.794
[2,   300] loss: 1.770
[2,   400] loss: 1.806
[2,   500] loss: 1.742
[2,   600] loss: 1.781
[2,   700] loss: 1.800
[2,   800] loss: 1.673
[3,   100] loss: 1.674
[3,   200] loss: 1.617
[3,   300] loss: 1.622
[3,   400] loss: 1.655
[3,   500] loss: 1.591
[3,   600] loss: 1.632
[3,   700] loss: 1.645
[3,   800] loss: 1.505
[4,   100] loss: 1.509
[4,   200] loss: 1.440
[4,   300] loss: 1.419
[4,   400] loss: 1.478
[4,   500] loss: 1.437
[4,   600] loss: 1.458
[4,   700] loss: 1.480
[4,   800] loss: 1.343
[5,   100] loss: 1.341
[5,   200] loss: 1.254
[5,   300] loss: 1.223
[5,   400] loss: 1.282
[5,   500] loss: 1.251
[5,   600] loss: 1.238
[5,   700] loss: 1.278
[5,   800] loss: 1.167
[6,   100] loss: 1.124
[6,   200] loss: 1.040
[6,   300] loss: 1.050
[6,   400] 

In [101]:
### Make predictions
pred = net(Variable(test_loader.dataset.data_tensor))
predicted_classes = torch.max(pred, 1)[1].data.numpy()
true_labels = test_loader.dataset.target_tensor.numpy()

In [102]:
### Print confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(true_labels, predicted_classes)

array([[134,  19,  31,  10,  14,   2,   6,  17,  70,   9],
       [ 14, 176,   7,  13,   8,   6,   7,   9,  30,  46],
       [ 21,   8, 102,  38,  57,  18,  36,  11,  17,   5],
       [  6,  19,  28, 109,  21,  47,  46,  15,  17,  12],
       [ 16,   6,  50,  23,  75,  16,  43,  35,  12,   4],
       [  5,  13,  30,  64,  19,  71,  33,  14,  13,   4],
       [  3,   9,  34,  47,  31,   6, 148,   9,  10,   8],
       [  8,  17,  23,  24,  33,  24,  18, 132,  10,  10],
       [ 35,  32,   3,   9,  16,   6,   3,   9, 184,  18],
       [ 21,  66,   3,  11,   8,   6,  10,  16,  28, 105]])